# **Fine Tuning Gemma Model**


In [1]:
!pip install LangChain
!pip install huggingface_hub
!pip install langchain_huggingface

In [2]:
!pip install datasets

In [3]:
!pip install -q transformers accelerate sentencepiece torch peft

In [4]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

In [5]:
# Choose Gemma model: "google/gemma-2b" or "google/gemma-7b"
model_id = "google/gemma-3-270m"

In [6]:
from huggingface_hub import notebook_login

notebook_login()

In [7]:
# Load tokenizer

tokenizer = AutoTokenizer.from_pretrained(model_id)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.16M [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.69M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/33.4M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/35.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

In [8]:
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",   # Automatically puts model on GPU if available
    torch_dtype=torch.bfloat16
)


config.json:   0%|          | 0.00/1.35k [00:00<?, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!


model.safetensors:   0%|          | 0.00/536M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/133 [00:00<?, ?B/s]

In [9]:
prompt = "Explain the concept of reinforcement learning in simple terms."
inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

In [10]:
# Generate output
outputs = model.generate(**inputs, max_new_tokens=200)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Explain the concept of reinforcement learning in simple terms.

​

-  ​A reward for behaviour and/or performance is given to the individual at the end of a period of learning behaviour. 

​

-  A reward for behaviour is an action or response that the individual provides the reward. 

​

-  When a response is rewarding, reinforcement is given to the individual. 

​

-  Reinforcement should be possible when the person has the opportunity to engage with the behaviour and perform the task. 

​

-  The person can continue to engage with the behaviour and perform the task or at least perform it to their satisfaction. 

​

-  The person feels that the behaviour is working to their benefit, which is rewarded by the reward. 

​

-  The person is rewarded by the reward. 

​

-  The behaviour is well-suited to the task. 

​

-  The person feels that the behaviour is not working, either to their benefit


In [11]:
# Apply LoRA for PEFT
from peft import LoraConfig, get_peft_model, TaskType
peft_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    r=16,                 # Rank
    lora_alpha=32,
    lora_dropout=0.1,
    bias="none"
)
model = get_peft_model(model, peft_config)

In [12]:
model= get_peft_model(model, peft_config)

/usr/local/lib/python3.12/dist-packages/peft/mapping_func.py:73: UserWarning: You are trying to modify a model with PEFT for a second time. If you want to reload the model with a different config, make sure to call `.unload()` before.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/peft/mapping_func.py:79: UserWarning: The PEFT config's `base_model_name_or_path` was renamed from 'google/gemma-3-270m' to 'None'. Please ensure that the correct base model is loaded when loading this checkpoint.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/peft/tuners/tuners_utils.py:196: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(


In [13]:
from datasets import load_dataset

dataset = load_dataset(
    "json",
    data_files="/content/drive/MyDrive/gemma_flat.json",
    split="train"
)

# Split into train/test
dataset = dataset.train_test_split(test_size=0.1, seed=42)

print(dataset)


Generating train split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['domain', 'category', 'question', 'answer'],
        num_rows: 162
    })
    test: Dataset({
        features: ['domain', 'category', 'question', 'answer'],
        num_rows: 18
    })
})


In [35]:
def format_example(example):
    batch_input_ids = []
    batch_attention_mask = []
    batch_labels = []

    for i in range(len(example['question'])):
        prompt = f"Instruction: Answer the following {example['domain'][i]} {example['category'][i]} question.\n"
        prompt += f"Input: {example['question'][i]}\nResponse:"

        # Tokenize input (prompt)
        input_tokens = tokenizer(
            prompt,
            truncation=True,
            padding="max_length",
            max_length=512
        )

        # Tokenize output (answer)
        label_tokens = tokenizer(
            example["answer"][i],
            truncation=True,
            padding="max_length",
            max_length=512
        )

        batch_input_ids.append(input_tokens["input_ids"])
        batch_attention_mask.append(input_tokens["attention_mask"])

        # Replace pad tokens in labels with -100 so they don’t affect loss
        labels = label_tokens["input_ids"]
        labels = [tok if tok != tokenizer.pad_token_id else -100 for tok in labels]
        batch_labels.append(labels)

    return {
        "input_ids": batch_input_ids,
        "attention_mask": batch_attention_mask,
        "labels": batch_labels
    }


In [36]:
# 4. Apply tokenizer to dataset
tokenized_ds = dataset.map(format_example, batched=True, remove_columns=dataset["train"].column_names)

Map:   0%|          | 0/162 [00:00<?, ? examples/s]

Map:   0%|          | 0/18 [00:00<?, ? examples/s]

In [37]:
from transformers import DataCollatorForSeq2Seq

# Data collator
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

In [38]:
from transformers import TrainingArguments
# Training args
training_args = TrainingArguments(
    output_dir="./gemma-interview-finetuned",
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    gradient_accumulation_steps=8,
    learning_rate=2e-4,
    num_train_epochs=3,
    logging_steps=10,
    save_strategy="epoch",
    eval_strategy="epoch",
    fp16=True,
    report_to="none"
)

In [39]:
from transformers import Trainer

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_ds["train"],
    eval_dataset=tokenized_ds["test"],
    tokenizer=tokenizer,
    data_collator=data_collator
)
trainer.train()

/tmp/ipython-input-2988009875.py:4: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss
1,13.852400,13.087485
2,12.487700,11.338896
3,11.005300,10.620934


TrainOutput(global_step=33, training_loss=12.301475409305457, metrics={'train_runtime': 64.0711, 'train_samples_per_second': 7.585, 'train_steps_per_second': 0.515, 'total_flos': 150886692421632.0, 'train_loss': 12.301475409305457, 'epoch': 3.0})

In [46]:
model.save_pretrained("/content/drive/MyDrive/gemma-interview-finetuned")
tokenizer.save_pretrained("/content/drive/MyDrive/gemma-interview-finetuned")

('/content/drive/MyDrive/gemma-interview-finetuned/tokenizer_config.json',
 '/content/drive/MyDrive/gemma-interview-finetuned/special_tokens_map.json',
 '/content/drive/MyDrive/gemma-interview-finetuned/tokenizer.model',
 '/content/drive/MyDrive/gemma-interview-finetuned/added_tokens.json',
 '/content/drive/MyDrive/gemma-interview-finetuned/tokenizer.json')